## Self-Attention mechanism

Reference:

https://arxiv.org/abs/1706.03762

https://github.com/sooftware/attentions

https://github.com/greentfrapp/attention-primer

### Brief introduction

The self-attention is one of the most popular and widely used mechanisms in the natural language processing area (NLP). 

Complex recurrent models are often used to capture the time dependency within the stimulus sequences (for example, phrases, sentences) to ensure the machine reaches a human-level performance in processing and generating languages. Most classic recurrent models with seq2seq training schema (GRU and LSTM) manage the input sequence using a simple philosophy: the input with a longer distance from the current stimulus must have less impact. The training algorithm--backpropagation through time (BPTT)--computes a dramatically lower gradient to the distanced input than the neighborhood. 

The self-attention mechanism processes the inputs depending on the inputs' representation similarity rather than time distance. The mechanism first estimates the between-input similarity and utilizes this similarity to weight these inputs, according to which it distributes its learning biases (via scaling the gradient). 

Here comes two implementational-level questions: 

1. How to calculate the similarity?
2. How to combine the computed attention with the input?

#### Similarity

In 1_RSA, we had introduced a few similarity (distance) metrics. Here, we use the correlation.

$$r(X,Y) = \frac{1}{n-1}\sum^n_i \frac{(x_i-\bar{x})(y_i-\bar{y})}{s_x s_y}$$ 
where $\bar{x}=\frac{1}{n}\sum_i^n x_i$, $s_x = \sqrt{\frac{1}{n-1}\sum_i^n (x_i-\bar{x})^2}$

Let's simplify this equation a bit. Assuming the elements in x, y are sampled from a Gaussian distribution $N(0, 1)$, we can remove the mean term $\bar{x}=0,\bar{y}=0$ and standard deviation $s_xs_y=1$.The correlation becomes a dot product of two vectors divided by the item number:

$$r'(X,Y) = \frac{1}{n-1}\sum^n_i x_i y_i = XY^{\top}$$
using the convention $X \in R^{1\times N}, X \in R^{1\times N}$ 

When using the attention mechanism, the model input is usually a sequence of vector $S = \{s^1, s^2, ...\}$, where the superscript indicates the location of the vector within a sequence. We can conduct a pairwise similarity of this sequence. The "Attention is all you need" paper dubbed the first element in each pair as a query, $Q$ and the second element as a key, $K$. The similarities between an arbitrary query and all keys are, 

$$R(j) = \frac{Q^jK^{\top}}{n}$$
where $-1$ is always neglected. Because $K \in N(0,1)$ and each $R(j)$ is approximately an linear combination of $R(j) \approx \frac{1}{n}K$, the variance of $R(j)$ is about $\frac{1}{\sqrt{n}}$. The similarity is then passed through a softmax function to form an attention distribution. 

$$A(j) = \text{softmax}\left(\frac{Q^jK^{\top}}{\sqrt{n}}\right)$$

Question here: why not $/n$?

Repeat the attention calculation for each query, we get a $A$. 

### Combine the attention and the input

First we need to know what is $Q$ and $K$? Both are linear transformation of the input $S$,

$$Q = S W_q$$
$$K = S W_k$$ 
where $S\in R^{N\times E}$, $N$ is the length of the input sequence, $E$ is the embedding dimension. $W_q, W_k \in R^{E\times H}$, h is the hidden layer dimension. $Q, K \in R^{H\times E}$, and $A \in R^{H \times H} = QK^{\top}$, 

Meanwhile, the input multiplies a matrix $V = S W_v, V \in R^{H \times V}$ for the simple reason of dimension matching.

Attended input is $S' \in R^{H\times V}= AV$


### A simple example 